![StaSteelExample1.png](\Resources\StaSteelExample1.png)
![StaSteelExample1.png](\Resources\StaSteelExample1_metaldeck.png)
![StaSteelExample1.png](\Resources\StaSteelExample1_metaldeck_load.png)
![StaSteelExample1.png](\Resources\StaSteelExample1_stud.png)
![StaSteelExample1.png](\Resources\StaSteelExample1_stud2.png)

In [41]:
import CompositeBeam as cp
import math

### Variables

In [42]:
# Birimler N,mm
comp = cp.CompositeBeams(# Yapım aşaması yükleri
DeadLoad   = 3.221, #kN/m^2
LiveLoad_c = 1, #kN/m^2

# Kompozit çalışma aşaması yükleri
SuperDeadLoad = 0.0, #kN/m^2
LiveLoad      = 2.0, #kN/m^2
w_girder      = 1.0, #kN/m kiriş ağırlığı

# Beton bilgisi
f_ck     = 20,     #N/mm2
wc       = 2500,   #kg/m3

# Çelik malzeme bilgisi
Fy       = 235,        #N/mm2
Fu_beam  = 235*1.2,    #N/mm2

# Çelik enkesit özellikleri
Abeam    = 2868.8,     #mm^2
Hbeam    = 200,        #mm
Hweb     = 378.8,      #mm
b_flange = 100,        #mm
t_flange = 8.5,        #mm
t_web    = 5.6,        #mm
Ix       = 1940*10**4, #mm^4
Ycon     = 150,        #mm Çelik kesit üst başlığından en üst beton lifine olan mesafe

# Kiriş uzunluk ve aralık bilgisi
Laralık  = 1250,  #mm
Lbeam    = 4840,  #mm

# Stud çivisi bilgisi
D_stud     = 19,      #mm
H_stud     = 100,     #mm
b_studhead = 32,      #mm
t_studhead = 10,      #mm
Fy_stud    = 235,     #N/mm2 çelik ankrajın minimum(karakteristik) akma dayanımı
Fu_stud    = 235*1.2, #N/mm2 çelik ankrajın minimum(karakteristik) çekme dayanımı
Nw         = 2,       #Kesit başlığının genişliğinde atılan stud çivisi adeti
IsWebAlignmentWelded=False, #Stud çivisi kiriş gövdesi izasında mı kaynaklanıyor.

# Metal sac bilgisi
hr       = 50,     #mm
wr       = 100,    #mm
t_sac    = 1.0,    #mm
IsParallel=True)

### Design Code Requirements

In [43]:
comp.ConcAvailableStressCheck(f_ck=f_ck)
comp.SteelAvailableStressCheck(Fy=Fy)
comp.MetalDeckCheck(hr=hr,wr=wr)
comp.ConcreteCoverAboveTopOfHeadedStudAnchorsChecks(hr=hr,t_sac=t_sac, h_stud=H_stud, Ycon=Ycon)
comp.StudCheck(Ds=D_stud, tf=t_flange, Hs=H_stud, IsWebAlignmentWelded=False)

20 N/mm^2 ≤ 20 N/mm^2 < 70 N/mm^2 √ - TSSDC 12.2.3(a)

235 N/mm^2 ≤ 460 N/mm^2 √ - TSSDC 12.2.3(c)

hr = 50mm ≤ 75mm √ - TSSDC 12.4.2.3

wr = 100mm ≥ 50mm √ - TSSDC 12.4.2.3

h_stud-hr-t_sac = 49.0mm ≥ 38mm √ - TSSDC 12.4.2.3

Ycon - h_stud-t_sac = 49.0mm ≥ 13mm √ - TSSDC 12.4.2.3

Ycon - hr = 100mm ≥ 50mm √ - TSSDC 12.4.2.3

Ds = 19mm ≤ 2.5*tf = 21.25mm √ - TSSDC 12.8.1

Ds = 19mm ≤ 19mm √ - TSSDC 12.8.1

Hs = 100mm ≥ 4*Ds = 76mm √ - TSSDC 12.8.2



True

### Yapım aşaması deplasman kontrolü

In [44]:
# Yapım aşaması
W_G = comp.Calc_Load(Load=DeadLoad, GirdersSpaceLength=Laralık/10**3) + w_girder
W_Q = comp.Calc_Load(Load=LiveLoad_c, GirdersSpaceLength=Laralık/10**3)
W_G,W_Q

(5.02625, 1.25)

In [45]:
# Yapım aşaması sabit yüklerinden(döşeme ve çelik kiriş) kaynaklı deplasman
delta_cdl = comp.SimpleCompositeBeamDeflection(w=W_G, Lbeam=Lbeam, I=Ix)
delta_cdl

9.26

In [46]:
print(f"{delta_cdl} < {(Lbeam/360)}")

9.26 < 13.444444444444445


In [47]:
# Ters sehim miktarı
delta_c = comp.Camber(delta_cdl=delta_cdl, Lbeam=Lbeam)

delta_cdl = 9.26mm < 19mm or Lbeam = 4840 < 7600mm olduğu için ters sehime gerek yoktur.



In [48]:
# Yapım aşaması hareketli yüklerinden kaynaklı deplasman
delta_yll = comp.SimpleCompositeBeamDeflection(w=W_Q, Lbeam=Lbeam, I=Ix)
delta_yll

2.3

In [49]:
# Yapım aşaması toplam düşey yerdeğiştirme, super dead düşey yerdeğiştirmesi yapım öncesi olmadığı için 0.0
delta_ytl = comp.calc_Delta_TL(delta_cdl=delta_cdl, delta_sdl=0., delta_ll=delta_yll,delta_c=delta_c)
delta_ytl

4.61

In [50]:
delta_ytl < Lbeam/240

True

### Kompozit kirişin karakteristik eğilme momenti dayanımının belirlenmesi

In [51]:
# Kiriş enkesit kontrolü
alfa = Hbeam/t_web
print(f"alfa = {round(alfa,2)} ≤ 3.76*(E/Fy)^0.5 = {round(3.76*(2*10**5/Fy)**0.5,2)} √")

alfa = 35.71 ≤ 3.76*(E/Fy)^0.5 = 109.69 √


In [52]:
# Effektif döşeme genişliği
b_eff = comp.EffectiveSlabWidth(L=Lbeam, Lu=Laralık)
b_eff

b_eff = 1210.0mm



1210.0

In [53]:
# Stud çivisiz beton yüksekliği
tc = comp.Calc_tc(hr=hr,Ycon=Ycon,IsParallel=True)
tc

tc = 125.0mm


125.0

In [54]:
# Stud çivisiz beton alanı
Ac = b_eff * tc
Ac

151250.0

In [55]:
Cs = comp.YieldBeamCapacity(fy=Fy,As=Abeam)
Cs

674168.0

In [56]:
Cc = comp.CrushConcCapacity(fck=f_ck,Ac=Ac)
Cc

2571250.0

In [57]:
Mn_design = comp.CompositeBeamDesignFlexuralCapacity(C_conc=Cc, T_steel=Cs, Ycon=Ycon, 
                                    BeamAs=Abeam, BeamFy= Fy, Hbeam=Hbeam, t_flange=t_flange, b_flange=b_flange, 
                                    Ac = Ac, hr=hr,fck=f_ck, beff=b_eff, tc=tc, t_web=t_web)
Mn_design/10**6

a = 32.77433155080214 mm
Y2 = 133.61283422459894 mm
Kompozit kiriş tam etkileşimlidir. Plastik tarafsız eksen beton döşemenin içindedir.
φMn = φ * (min(C_conc,T_steel) * ( 0.5*Hbeam + hr + tc - 0.5*a)) 
= 0.9 * 674168.0 *  258.61 = 156913647.501


156.9136475

In [58]:
Cf        = comp.Calc_Cf(Vbeam=Cs, Vcon=Cc)
a         = comp.Calc_a(Cf =Cf, fck = comp.f_ck, b_eff = b_eff)
Y2        = comp.Calc_Y2(Ycon =comp.Ycon, a =a)
C_flange  = comp.Calc_Cflange(t_flange=comp.t_flange, b_flange = comp.b_flange, BeamFy=comp.Fy)
Y1        = comp.Calc_Y1(T_steel=Cs, C_conc=Cc, C_flange=C_flange, t_flange=comp.t_flange, t_web=comp.t_web, BeamFy=comp.Fy)
C_web     = comp.Calc_Cweb(Y1=Y1, t_flange=comp.t_flange, t_web=comp.t_web, BeamFy=comp.Fy)
Mn_web    = comp.PTEInWebMn(T_steel=Cs, C_conc=Cc, Cflange=C_flange, Cweb=C_web, Hbeam=comp.Hbeam, t_flange=comp.t_flange, Y1=Y1, Y2=Y2)
Mn_flange = comp.PTEInFlangeMn(C_conc=Cc, Cflange=C_flange, Hbeam=comp.Hbeam, BeamAs=comp.Abeam, BeamFy=comp.Fy, Y1=Y1, Y2=Y2)
Mn_slab   = comp.PTEInSlabMn(Cf=Cf, Hbeam=comp.Hbeam, hr=comp.hr, tc=tc, a=a)
Mn_design = comp.CompositeBeamDesignFlexuralCapacity2(C_conc=Cc, T_steel=Cs,Mn_web=Mn_web, Mn_slab=Mn_slab, Mn_flange=Mn_flange, Y1=Y1, t_flange=comp.t_flange, fi_b=0.9)

W_superdeadload = comp.Calc_Load(Load=comp.SuperDeadLoad,GirdersSpaceLength=comp.Laralık/10**3)
W_G_composite = W_G + W_superdeadload
W_Q_composite = comp.Calc_Load(Load=comp.LiveLoad, GirdersSpaceLength=comp.Laralık/10**3)
W_TL = comp.calc_w_tl(w_dl=W_G_composite, w_ll=W_Q_composite)
M_demand = W_TL * comp.Lbeam**2 /8
comp.CompositeBeamFlexuralCapacityCheck(M_demand=M_demand, Mn_design=Mn_design)

Ec = comp.ConcreteYoungModules(fck=comp.f_ck, wc=2500)
A_sa = 3.14 * comp.D_stud**2 /4
Rp = comp.GetRp(Hstud = comp.H_stud, hr =comp.hr, t_studhead = 5, IsPitchParaleltoBeam=True)
Rg = comp.GetRg(StudsNumberInFlangeWidth=2, wr=comp.wr, hr=comp.hr, IsPitchParaleltoBeam=True)
Qn = comp.OneStudShearCapacity(Asa=A_sa, fck=comp.f_ck, Ec=Ec, Rg=Rg, Rp=Rp, Fu=comp.Fu_beam)
N_studfullinteraction = math.ceil(Cf / Qn) * comp.Nw
# Metal sacın kirişe dik atıldığı durumda, stud çivisi sayısını hadve boşlukları ara mesafesi belirler. örneğin hadveler arası mesafe 305mm ve 9150mm uzunluğunda bir kirişe max 30 adet atılabilir. Tam etkileşimli durumda oluyorsa bu durum çift sıraya veya kiriş arttırımına gidilmek zorunluluğu ortaya çıkar.

TotalQn_fullinteraction = N_studfullinteraction*Qn
comp.DistanceBetweenTwoStudsCheck(x=30, Ds=comp.D_stud)
comp.StudSpaceCheck(Ds=comp.D_stud, s=115, Ycon=comp.Ycon)

TotalQn = N_studfullinteraction*Qn
n = comp.RatioYoungModules(Ec=Ec)
Act = comp.ChengedConcToSteelArea(n=n, Ac=Ac)
Itr = comp.Calc_I_tr(Act=Act, n=n, b_eff=b_eff, hr=comp.hr, tc=tc, hb=comp.Hbeam, Ab=comp.Abeam, Ibeam=comp.Ix) #Tam etkileşimli kompozit kiriş sehim kontrolü için kullanılır
Ieff = comp.Calc_I_eff(Ibeam=comp.Ix, TotalQn=TotalQn_fullinteraction, Cf=Cf, Itr=Itr)
I_real = Ieff = comp.Calc_I_real(I_eff=Ieff) #Kısmi etkileşimli kompozit kiriş sehim kontrolleri için kullanılır.


a = 32.77mm

Y2 = 133.615mm

Y1=-864.0615501519757 < 0 ==> Y1 = 0

Kompozit kiriş tam etkileşimlidir. Plastik tarafsız eksen beton döşemenin içindedir.

φMn = φ * Mn_slab = 0.9 * 174.353kNm = 156.918kNm

Talep/Kapasite oranı = 0.19

Rp = 0.75

Rg = 1.0

Qn = 0.5 * Asa * (fck*Ec)**0.5 = 98.244579kN

Qn = 98.244579kN > Rg*Rp*Asa*Fu = 1.0*0.75*283.385*282.0 = 59.93592700000001kN ==> Qn = 59.93592700000001kN

s = 115mm ≥ 114mm √

s = 115mm ≤ 1200mm √

s = 115mm ≤ 914mm √



### Arttırılmış düşey yükler altında eğilme momenti kapasite talebi

In [59]:
W_superdeadload = comp.Calc_Load(Load=comp.SuperDeadLoad,GirdersSpaceLength=comp.Laralık/10**3)
W_superdeadload

0.0

In [60]:
W_G_composite = W_G + W_superdeadload
W_G_composite

5.02625

In [61]:
W_Q_composite = comp.Calc_Load(Load=comp.LiveLoad, GirdersSpaceLength=comp.Laralık/10**3)
W_Q_composite

2.5

In [62]:
W_TL = comp.calc_w_tl(w_dl=W_G_composite, w_ll=W_Q_composite)
W_TL

10.031500000000001

In [63]:
# Talep eğilme momenti dayanımı
M_demand = W_TL * comp.Lbeam**2 /8
M_demand/10**6

29.374238300000005

### Talep/Kapasite oranı

In [64]:
flexuralcapacityratio = round(M_demand / Mn_design,2)
flexuralcapacityratio

0.19

### Başlıklı çelik ankrajların sayısı ve yerleşimi

In [65]:
Ec = comp.ConcreteYoungModules(fck=comp.f_ck, wc=2500)
Ec

24037.73075812274

In [66]:
A_sa = 3.14 * comp.D_stud**2 /4
A_sa

283.385

In [67]:
Rg = comp.GetRg(StudsNumberInFlangeWidth=2, wr=comp.wr, hr=comp.hr, IsPitchParaleltoBeam=comp.IsParallel)
Rg

Rg = 1.0



1.0

In [70]:
Rp = comp.GetRp(Hstud = comp.H_stud, hr =comp.hr, t_studhead = 5, IsPitchParaleltoBeam=comp.IsParallel)
Rp

Rp = 0.75



0.75

In [71]:
Qn = comp.OneStudShearCapacity(Asa=A_sa, fck=comp.f_ck, Ec=Ec, Rg=Rg, Rp=Rp, Fu=comp.Fu_beam)
Qn

Qn = 0.5 * Asa * (fck*Ec)**0.5 = 98.244579kN

Qn = 98.244579kN > Rg*Rp*Asa*Fu = 1.0*0.75*283.385*282.0 = 59.93592700000001kN ==> Qn = 59.93592700000001kN



59935.927

In [72]:
Cs,Cc

(674168.0, 2571250.0)

In [73]:
# Tam etkileşimli olduğu için
C = min(Cs,Cc)
C

674168.0

In [74]:
V_üs = C
V_üs

674168.0

In [75]:
#  Başlık 2 şerli olarak kullanılması öngörüldüğü için 2 ile çarpılarak maksimum moment ile eğilme momentinin sıfır olduğu en yakın nokta (mesnet) arasında kullanılacak çelik ankraj sayısı

N_studfullinteraction = math.ceil(V_üs / Qn) * 2
N_studfullinteraction

24

### Kısmi etkileşimli kabul edilip kompozitlik derecesi %25 alınırsa

In [76]:
TotalQn = C*0.25
TotalQn

168542.0

In [77]:
N_studpartialinteraction = math.ceil(TotalQn / Qn)*2
N_studpartialinteraction

6

In [78]:
Mn_design2 = comp.CompositeBeamDesignFlexuralCapacity(C_conc=TotalQn,T_steel=Cs,Ycon=comp.Ycon, 
                                    BeamAs=comp.Abeam, BeamFy= comp.Fy, Hbeam=comp.Hbeam, t_flange=comp.t_flange, b_flange=comp.b_flange, 
                                    Ac = Ac, hr=comp.hr,fck=comp.f_ck, beff=b_eff, tc=tc, t_web=comp.t_web)
Mn_design2/10**6

a = 8.193582887700535 mm
Y2 = 145.90320855614974 mm
Kompozit kiriş kısmi etkileşimlidir. Kompozitlik oranı %25'in altına inmemelidir.

Basınç bloğu başlıkta kabul edilirse; Y1(veya tf') = 10.76 mm
PTE kiriş gövdesindedir.Çelikteki basınç derinliği(Y1 veya tf') yeniden hesaplanacak...
Cflange = b_flange * t_flange * Fy = 199750.0 N
Y1 = ((T_steel - C_conc - 2*Cflange) / (2*t_web*Fy)) + t_flange = 48.82142857142857mm
Cweb = t_web * Fy * (Y1 - t_flange) = 53063.0 N
φMn = φ * (C_conc*(Y1+Y2) + 2*Cflange*(Y1 - 0.5*t_flange) + 2*Cweb*(0.5*(Y1-t_flange)) + T_steel*(0.5*Hbeam-Y1)) = 0.9 * [32819279.791 + 17806285.714 + 2139575.964 + 34502955.143] = 78541286.951 


78.54128695

In [79]:
M_demand/Mn_design2

0.3739974151263892

### Kesit atalet momentleri

In [80]:
# Elastisite oranları
n = comp.RatioYoungModules(Ec=Ec)
n

8.32

In [81]:
# Dönüştürülmüş alan
Act = comp.ChengedConcToSteelArea(n=n, Ac=Ac)
Act

18179.087

In [82]:
# Dönüştürülmüş kesitin atalet momenti
Itr = comp.Calc_I_tr(Act=Act, n=n, b_eff=b_eff, hr=comp.hr, tc=tc, hb=comp.Hbeam, Ab=comp.Abeam, Ibeam=comp.Ix)
Itr

154958220.48821288

In [83]:
# Effektif atalet moementi
Ieff = comp.Calc_I_eff(Ibeam=comp.Ix, TotalQn=TotalQn, Cf=Cf, Itr=Itr)
Ieff

87179110.24410644

In [84]:
# Gerçek atalet momenti
I_real = Ieff = comp.Calc_I_real(I_eff=Ieff)
I_real

65384332.68307983